# Insert and Delete Queries with `doctable`

In this document I will describe the interface for performing insert and delete queries with doctable.

In [17]:

import pandas as pd
import numpy as np
import typing

import sys
sys.path.append('..')
import doctable

#### Define a demonstration schema

The very first step is to define a table schema that will be appropriate for our examples. This table includes the typical `id` column (the first column, specified by `order=0`), as well as string, integer, and boolean attributes. The object used to specify the schema is called a _container_, and I will use that terminology as we go.

In [18]:
@doctable.table_schema
class Record:
    name: str = doctable.Column(column_args=doctable.ColumnArgs(nullable=False, unique=True))
    age: int = doctable.Column()
    is_old: bool = doctable.Column()
    
    id: int = doctable.Column(
        column_args=doctable.ColumnArgs(
            order = 0, 
            primary_key=True, 
            autoincrement=True
        ),
    )

core = doctable.ConnectCore.open(target=':memory:', dialect='sqlite', echo=True)

with core.begin_ddl() as ddl:
    rtab = ddl.create_table_if_not_exists(container_type=Record)

2023-11-13 09:12:38,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:12:38,149 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Record")
2023-11-13 09:12:38,150 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-13 09:12:38,153 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Record")
2023-11-13 09:12:38,154 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-13 09:12:38,159 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Record" (
	id INTEGER, 
	age INTEGER, 
	is_old INTEGER, 
	name VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (name)
)


2023-11-13 09:12:38,160 INFO sqlalchemy.engine.Engine [no key 0.00124s] ()
2023-11-13 09:12:38,162 INFO sqlalchemy.engine.Engine COMMIT


## Two Interfaces: `ConnectQuery` and `TableQuery`

First, a little about the `doctable` query interface. There are two interfaces for performing queries: `ConnectQuery` and `TableQuery`. 

+ **`ConnectQuery`** table-agnostic interface for querying any table in any result format. Create this object using the `ConnectCore.query()` method.

+ **`TableQuery`** table-specific interface for querying a specific table. Insert and select from container objects used to define the schema. Create this object using the `DBTable.query()` method.

### Inserts via `ConnectQuery`

First I will discuss the `ConnectQuery` interface, which is created via the `ConnectCore.query()` method. This object maintains a database connection, and, when used as a context manager, will commit all changes upon exit. It is fine to use the `ConnectQuery` object without a context manager for queries that do not require commits.

There are two primary methods for insertions via the `ConnectQuery` interface, which you can see in this table. Both accept a single `DBTable` object, followed by one or multiple dictionaries of data to insert, depending on the method.

| Method | Description |
| --- | --- |
| `insert_single()` | Insert a single row into a table. |
| `insert_multi()` | Insert multiple rows into a table. |

In [19]:
with core.query() as q:
    q.insert_single(rtab, {
        'name': 'test_A',
        'age': 10,
        'is_old': False,
    })
    
    q.insert_multi(rtab, data = [
        {
            'name': 'test_B',
            'age': 10,
            'is_old': False,
        },
        {
            'name': 'test_C',
            'age': 10,
            'is_old': False,
        }
    ])
    
q.select(rtab.all_cols()).df()

2023-11-13 09:12:38,210 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:12:38,212 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (age, is_old, name) VALUES (?, ?, ?)
2023-11-13 09:12:38,213 INFO sqlalchemy.engine.Engine [generated in 0.00349s] (10, False, 'test_A')
2023-11-13 09:12:38,217 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (age, is_old, name) VALUES (?, ?, ?)
2023-11-13 09:12:38,219 INFO sqlalchemy.engine.Engine [generated in 0.00164s] [(10, False, 'test_B'), (10, False, 'test_C')]
2023-11-13 09:12:38,221 INFO sqlalchemy.engine.Engine COMMIT
2023-11-13 09:12:38,224 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:12:38,226 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-13 09:12:38,227 INFO sqlalchemy.engine.Engine [generated in 0.00289s] ()


,id,age,is_old,name
0,1,10,0,test_A
1,2,10,0,test_B
2,3,10,0,test_C


#### Omit Attributes

If some values are not provided, the database will decide which values they take. In this case, the database populates the ID column according to the schema (it acts as the primary key in this case).

In [20]:
core.query().insert_single(rtab, {
    'name': 'test_D',
})
core.query().select(rtab.all_cols()).df()

2023-11-13 09:12:38,273 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:12:38,275 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (name) VALUES (?)
2023-11-13 09:12:38,277 INFO sqlalchemy.engine.Engine [generated in 0.00390s] ('test_D',)
2023-11-13 09:12:38,281 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:12:38,282 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-13 09:12:38,283 INFO sqlalchemy.engine.Engine [cached since 0.05923s ago] ()


,id,age,is_old,name
0,1,10.0,0.0,test_A
1,2,10.0,0.0,test_B
2,3,10.0,0.0,test_C
3,4,NaN,NaN,test_D


Note that in our schema we set `nullable=False` for the name column, so this must be provided in an insert otherwise there will be an error. This typically results in an `sqlalchemy.exc.IntegrityError`, which you may catch if needed.

In [30]:
import sqlalchemy.exc

try:
    core.query().insert_single(rtab, {
        'is_old': True,
    })
except sqlalchemy.exc.IntegrityError as e:
    print(type(e), e)

2023-11-13 09:19:59,660 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:19:59,661 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (is_old) VALUES (?)
2023-11-13 09:19:59,662 INFO sqlalchemy.engine.Engine [cached since 223.6s ago] (True,)
<class 'sqlalchemy.exc.IntegrityError'> (sqlite3.IntegrityError) NOT NULL constraint failed: Record.name
[SQL: INSERT OR FAIL INTO "Record" (is_old) VALUES (?)]
[parameters: (True,)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### `ifnotunique` Parameter

The `ifnotunique` paramter controls the behavior when a unique constraint is violated. 

The default value is `FAIL`, which will raise an error when a unique constraint is violated - it will raise an `sqlalchemy.exc.IntegrityError` exception in this case. The other options are `IGNORE`, meaning inserted rows that violate the constraints should be ignored, and `REPLACE`, which will replace the existing row with the new row.

In the `Record` table we have created, there is a unique constraint on `name`. We will receive an integrity error if we try to insert a duplicate there when using the default `ifnotunique='ERROR'`.

In [31]:
try:
    core.query().insert_single(rtab, {
        'name': 'test_A',
    })
except sqlalchemy.exc.IntegrityError as e:
    print(type(e), e)

2023-11-13 09:20:49,672 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:20:49,675 INFO sqlalchemy.engine.Engine INSERT OR FAIL INTO "Record" (name) VALUES (?)
2023-11-13 09:20:49,677 INFO sqlalchemy.engine.Engine [cached since 491.4s ago] ('test_A',)
<class 'sqlalchemy.exc.IntegrityError'> (sqlite3.IntegrityError) UNIQUE constraint failed: Record.name
[SQL: INSERT OR FAIL INTO "Record" (name) VALUES (?)]
[parameters: ('test_A',)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


When using `ifnotunique='REPLACE'`, the insert will replace the existing row with the new row. This is useful when you want to update a row if it already exists, but insert it if it does not.

In [34]:
core.query().insert_single(rtab, {
    'name': 'test_A',
}, ifnotunique='REPLACE')
core.query().select(rtab.all_cols()).df()

2023-11-13 09:24:22,472 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:24:22,474 INFO sqlalchemy.engine.Engine INSERT OR REPLACE INTO "Record" (name) VALUES (?)
2023-11-13 09:24:22,475 INFO sqlalchemy.engine.Engine [cached since 74.85s ago] ('test_A',)
2023-11-13 09:24:22,477 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-13 09:24:22,478 INFO sqlalchemy.engine.Engine SELECT "Record".id, "Record".age, "Record".is_old, "Record".name 
FROM "Record"
2023-11-13 09:24:22,479 INFO sqlalchemy.engine.Engine [cached since 704.3s ago] ()


,id,age,is_old,name
0,2,10.0,0.0,test_B
1,3,10.0,0.0,test_C
2,4,NaN,NaN,test_D
3,7,NaN,NaN,test_A


In [21]:
test_record = Record(name='test', age=10, is_old=False)
test_record

Record(name='test', age=10, is_old=False, id=MISSING)